<a href="https://colab.research.google.com/github/FlorianSong/MResAMS_DataAnalytics/blob/main/Workshop2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data analytics 2022
### Workshop 2 &ndash; Machine Learning Classics: Supervised Learning &ndash; 2rd November 2022
##### Taught by: Nan Wu, Wilson Wu, Florian Song, Linden Schrecker, Annabel Basford, Sophia Yaliraki

Much of today's workshop was taken from https://github.com/ageron/handson-ml2/ which in turn is based on the second edition of an O'Reilly book [Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) by Aurélien Geron.

In [15]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
mpl.rc('axes', labelsize = 14)
mpl.rc('xtick', labelsize = 12)
mpl.rc('ytick', labelsize = 12)


# all the functions from sklearn needed for this workshop
import sklearn
from sklearn.datasets import make_moons, make_circles, load_iris, load_wine
from sklearn.svm import LinearSVC, SVC, SVR
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Take-home challenge: Classifying biodegradability

- This week, we will look at the QSAR biodegradation dataset available [here](https://archive.ics.uci.edu/ml/datasets/QSAR+biodegradation) which can also be found in the datasets folder in this workspace. This dataset contains 41 molecular descriptors and 1 experimental class: ready biodegradable (RB) and not ready biodegradable (NRB). As you can tell, this dataset lends itself nicely to using decision trees and random forests. 
- Firstly, have a look at the data webpage linked above. Familiarise yourself with all the descriptors and think about which ones may have more predicting power over others. 
- Using a package of your choice (`csv` or `pandas` or something else), read in the dataset and encode the class variable as integers. The direct link to the dataset is here (this works directly with pandas.read_csv!): `https://archive.ics.uci.edu/ml/machine-learning-databases/00254/biodeg.csv` 
- Fit a decision tree (*training/test*!). What's the prediction success rate? Find out the spread of the success rate (mean, variance).
- Fit a random forest. Does this improve things? 
- Can you tune the parameters to get an even better prediction rate? 
- Which parameters are more important than others? What happens when you train on a subset of parameters? Play around with this to see what happens. 
- *Bonus:* In the associated paper, the authors used - amongst other methods - Support Vector Machines fitted to the data. Read to the extra material on SVMs and try to fit an SVM to this data. How does it perform compared to the random forest?  
- *Bonus 2:* Implement a simple stacking ensemble method with your choice of predictors. Again, can you improve your high-score?
- *Bonus 3:* Using at least 5-fold cross-validation (https://scikit-learn.org/stable/modules/cross_validation.html) to obtain a range of predictions, report your average precision for your best model.



In [2]:
# Data loading and encoding the outcome variable

import numpy as np
import pandas as pd

biodeg = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00254/biodeg.csv', 
                     sep = ';', header = None)
biodeg = biodeg.to_numpy()

X_biodeg = biodeg[:, :-1].astype(float)
y_biodeg = biodeg[:, -1].astype(str)

y_biodeg = np.array([0 if i == 'NRB' else 1 for i in y_biodeg]).astype(int)

In [21]:
# Fitting a decision tree and reporting a success score

X_biodeg_train, X_biodeg_test, y_biodeg_train, y_biodeg_test = train_test_split(X_biodeg, y_biodeg)
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_biodeg_train, y_biodeg_train)
decision_tree.score(X_biodeg_test, y_biodeg_test)

0.803030303030303

In [22]:
# Record the scores of 500 Decision trees and get the mean and standard deviation of all scores

scores = []
for i in range(500):
    X_biodeg_train, X_biodeg_test, y_biodeg_train, y_biodeg_test = train_test_split(X_biodeg, y_biodeg)
    scores += [DecisionTreeClassifier().fit(X_biodeg_train, y_biodeg_train).score(X_biodeg_test, y_biodeg_test)]
print(np.mean(scores), np.std(scores))

0.809719696969697 0.022391216209953463


In [24]:
# Fit a random forest to the data

X_biodeg_train, X_biodeg_test, y_biodeg_train, y_biodeg_test = train_test_split(X_biodeg, y_biodeg)
random_forest = RandomForestClassifier(n_estimators = 500)
random_forest.fit(X_biodeg_train, y_biodeg_train)
    
random_forest.score(X_biodeg_test, y_biodeg_test)

0.8484848484848485

In [25]:
# Run 50 random forests, each with 100 trees, to get the average score and std deviation

scores = []
for i in range(50):
    X_biodeg_train, X_biodeg_test, y_biodeg_train, y_biodeg_test = train_test_split(X_biodeg, y_biodeg)
    scores += [RandomForestClassifier(n_estimators = 100).fit(X_biodeg_train, y_biodeg_train).score(X_biodeg_test, y_biodeg_test)]
print(np.mean(scores), np.std(scores))

0.8686363636363638 0.017282692500160266


In [26]:
# Get information about the importance of each predictor variable
# Header names were taken from the paper (downloadable excel file)

header_names = np.array( ['SpMax_L', 'J_Dz(e)', 'nHM', 'F01[N-N]', 'F04[C-N]', 'NssssC', 'nCb-', 'C%', 'nCp', 'nO', 'F03[C-N]', 'SdssC', 'HyWi_B(m)', 'LOC', 'SM6_L', 'F03[C-O]', 'Me', 'Mi', 'nN-N', 'nArNO2', 'nCRX3', 'SpPosA_B(p)', 'nCIR', 'B01[C-Br]', 'B03[C-Cl]', 'N-073', 'SpMax_A', 'Psi_i_1d', 'B04[C-Br]', 'SdO', 'TI2_L', 'nCrt', 'C-026', 'F02[C-N]', 'nHDon', 'SpMax_B(m)', 'Psi_i_A', 'nN', 'SM6_B(m)', 'nArCOOR', 'nX'])
X_biodeg_train, X_biodeg_test, y_biodeg_train, y_biodeg_test = train_test_split(X_biodeg, y_biodeg)
random_forest = RandomForestClassifier(n_estimators = 1000).fit(X_biodeg_train, y_biodeg_train)
ordered = random_forest.feature_importances_.argsort()[::-1]
for (i, name, score) in zip(ordered, header_names[ordered], random_forest.feature_importances_[ordered]):
    print('{}th feature:'.format(str(i).rjust(2)) , str(round(score,3)).rjust(5),  name)

35th feature: 0.092 SpMax_B(m)
 0th feature: 0.074 SpMax_L
21th feature: 0.064 SpPosA_B(p)
38th feature: 0.059 SM6_B(m)
26th feature: 0.056 SpMax_A
14th feature: 0.041 SM6_L
12th feature: 0.035 HyWi_B(m)
11th feature: 0.035 SdssC
29th feature: 0.032 SdO
36th feature: 0.032 Psi_i_A
15th feature: 0.031 F03[C-O]
17th feature: 0.031 Mi
 1th feature:  0.03 J_Dz(e)
 7th feature: 0.029 C%
13th feature: 0.029 LOC
33th feature: 0.028 F02[C-N]
30th feature: 0.028 TI2_L
 9th feature: 0.027 nO
37th feature: 0.027 nN
16th feature: 0.027 Me
 2th feature: 0.023 nHM
27th feature:  0.02 Psi_i_1d
10th feature: 0.019 F03[C-N]
 4th feature: 0.018 F04[C-N]
 8th feature: 0.016 nCp
 5th feature: 0.015 NssssC
 6th feature: 0.015 nCb-
32th feature: 0.013 C-026
40th feature: 0.012 nX
34th feature: 0.012 nHDon
39th feature: 0.009 nArCOOR
22th feature: 0.008 nCIR
31th feature: 0.004 nCrt
24th feature: 0.003 B03[C-Cl]
 3th feature: 0.002 F01[N-N]
19th feature: 0.001 nArNO2
18th feature: 0.001 nN-N
25th feature: 0.

In [27]:
# Get the top n predictors and train only with those

top_predictors = random_forest.feature_importances_.argsort()[-17:][::-1]
X_biodeg_new = X_biodeg[:, top_predictors]

X_biodeg_train_new, X_biodeg_test_new, y_biodeg_train_new, y_biodeg_test_new = train_test_split(X_biodeg_new, y_biodeg)

new_random_forest = RandomForestClassifier(n_estimators = 500)
new_random_forest.fit(X_biodeg_train_new, y_biodeg_train_new)

new_random_forest.score(X_biodeg_test_new, y_biodeg_test_new)

0.8636363636363636

In [28]:
# Bonus 1: Fitting an SVM

svm_rbf = Pipeline([
        ('scaler', StandardScaler()),
        ('svm_clf', SVC(kernel = 'rbf'))
    ])

svm_rbf.fit(X_biodeg_train, y_biodeg_train)

y_biodeg_pred_svm = svm_rbf.predict(X_biodeg_test)
accuracy_score(y_biodeg_pred_svm, y_biodeg_test)

0.8522727272727273

In [10]:
# Bonus 2: Stacked ensemble learning (building from scratch)

X_train_val, X_test, y_train_val, y_test = train_test_split(X_biodeg, y_biodeg)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val)
print(X_biodeg.shape, X_train.shape, X_val.shape, X_test.shape)

random_forest_clf = RandomForestClassifier(n_estimators = 100)
extra_trees_clf = ExtraTreesClassifier(n_estimators = 100)
svm_clf = Pipeline([
        ('scaler', StandardScaler()),
        ('svm_clf', SVC(kernel = 'rbf', gamma = 'auto'))
    ])
mlp_clf = MLPClassifier(max_iter = 1000)


estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print('Training the', estimator)
    estimator.fit(X_train, y_train)
    
print('Accuracies:', [estimator.score(X_val, y_val) for estimator in estimators])

X_val_predictions = np.empty((len(X_val), len(estimators)), dtype = np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)
        
        
rnd_forest_blender = RandomForestClassifier(n_estimators = 200, oob_score = True)
rnd_forest_blender.fit(X_val_predictions, y_val)


X_test_predictions = np.empty((len(X_test), len(estimators)), dtype = np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)
    
y_pred = rnd_forest_blender.predict(X_test_predictions)

accuracy_score(y_test, y_pred)

(1055, 41) (593, 41) (198, 41) (264, 41)
Training the RandomForestClassifier()
Training the ExtraTreesClassifier()
Training the Pipeline(steps=[('scaler', StandardScaler()), ('svm_clf', SVC(gamma='auto'))])
Training the MLPClassifier(max_iter=1000)
Accuracies: [0.8737373737373737, 0.8939393939393939, 0.8686868686868687, 0.8686868686868687]


0.8787878787878788

In [29]:
# Bonus 2: Stacked ensemble learning (simple stacking)

from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators = [
        ('rf', RandomForestClassifier(n_estimators = 100, random_state = 42)),
        ('etr', ExtraTreesClassifier(n_estimators = 100, random_state = 42)),
        ('svm', Pipeline([('scaler', StandardScaler()), ('svm_clf', SVC(kernel = 'rbf', gamma = 'auto'))])),
        ('mlp', MLPClassifier(max_iter = 1000, random_state = 42))
    ],
    final_estimator = RandomForestClassifier(n_estimators = 200, oob_score = True, random_state = 42)
)

stacking_clf.fit(X_biodeg_train, y_biodeg_train)

StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('etr', ExtraTreesClassifier(random_state=42)),
                               ('svm',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('svm_clf',
                                                 SVC(gamma='auto'))])),
                               ('mlp',
                                MLPClassifier(max_iter=1000, random_state=42))],
                   final_estimator=RandomForestClassifier(n_estimators=200,
                                                          oob_score=True,
                                                          random_state=42))

In [30]:
y_biodeg_pred = stacking_clf.predict(X_biodeg_test)

accuracy_score(y_biodeg_test, y_biodeg_pred)

0.8484848484848485

In [31]:
# Bonus 3: Compare a range of models to get the best 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier

estimators = {
    'Nearest Neighbors': KNeighborsClassifier(2), 
    'RBF SVM' : Pipeline([
        ('scaler', StandardScaler()),
        ('svm_clf', SVC(kernel = 'rbf', gamma = 'auto'))
    ]),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators = 100),
    'Extremely randomized trees': ExtraTreesClassifier(n_estimators = 100), 
    'Gradient Boost': GradientBoostingClassifier(),
    'Neural Net': MLPClassifier(max_iter = 1000),
    'AdaBoost': AdaBoostClassifier(),
    'Naive Bayes': GaussianNB(), 
    'Voting soft (ERT, SVM, NN)': VotingClassifier(estimators = [
        ('ERT', ExtraTreesClassifier(n_estimators = 100)), 
        ('SVM', Pipeline([
            ('scaler', StandardScaler()),
            ('svm_clf', SVC(kernel = 'rbf', gamma = 'auto', probability = True))
        ])), 
        ('NN', MLPClassifier(max_iter=1000)),
    ], voting = 'soft'), 
    'Stacking (ERT, SVM, NN)': StackingClassifier(estimators = [
        ('ERT', ExtraTreesClassifier(n_estimators = 100)),
        ('SVM', Pipeline([
            ('scaler', StandardScaler()),
            ('svm_clf', SVC(kernel = 'rbf', gamma = 'scale', probability = True))
        ])),
        ('NN', MLPClassifier(max_iter = 1000))
    ], final_estimator = RandomForestClassifier(n_estimators = 500))

}
for name,estimator in estimators.items():
    scores = cross_val_score(estimator, X_biodeg, y_biodeg, cv = 5)
    print(name.ljust(30) + 'Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Nearest Neighbors             Accuracy: 0.81 (+/- 0.09)
RBF SVM                       Accuracy: 0.87 (+/- 0.08)
Decision Tree                 Accuracy: 0.80 (+/- 0.05)
Random Forest                 Accuracy: 0.85 (+/- 0.10)
Extremely randomized trees    Accuracy: 0.85 (+/- 0.09)
Gradient Boost                Accuracy: 0.85 (+/- 0.11)
Neural Net                    Accuracy: 0.86 (+/- 0.07)
AdaBoost                      Accuracy: 0.85 (+/- 0.11)
Naive Bayes                   Accuracy: 0.71 (+/- 0.06)
Voting soft (ERT, SVM, NN)    Accuracy: 0.88 (+/- 0.07)
Stacking (ERT, SVM, NN)       Accuracy: 0.87 (+/- 0.08)
